# Tutorial 7: The Typed Passage Discipline

*Brennis Mund's Final Synthesis*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-typed-passages/blob/main/notebooks/07-the-typed-passage-discipline.ipynb)

---

## A Note from the Archives

> *Year 858, Capital Archives*
>
> Brennis Mund catalogued his final expression: $\lambda x:\text{nat}.x$ — the typed identity on naturals.
>
> It was a small thing, this final contribution. A simple passage that returned its argument unchanged. His father Kelleth had written the untyped version seventy years earlier. But this version carried something new: a boundary. A declaration of what it would accept.
>
> "The calculus is smaller now," Brennis wrote in his retirement notes. "But within its boundaries, nothing goes wrong. Every passage terminates. Every application succeeds. The Archive is safe."
>
> His epitaph would read: **"He gave passages their boundaries."**

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. State the complete simply typed lambda calculus
2. Summarize the key theorems and their implications
3. Apply the typing discipline to classify expressions
4. Evaluate the tradeoffs Brennis made
5. Connect to the broader Types & Computation narrative

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions_df = pd.read_csv(BASE_URL + "typed_passage_expressions.csv")
comparison_df = pd.read_csv(BASE_URL + "type_system_comparison.csv")
derivations_df = pd.read_csv(BASE_URL + "typing_derivations.csv")
traces_df = pd.read_csv(BASE_URL + "normalization_traces.csv")

print("Loaded all datasets for final synthesis")

## 1. The Complete System

### Syntax

**Types:**
$$\tau ::= \text{nat} \mid \text{bool} \mid \tau_1 \to \tau_2$$

**Expressions:**
$$M ::= x \mid \lambda x:\tau.M \mid M_1\ M_2$$

**Contexts:**
$$\Gamma ::= \emptyset \mid \Gamma, x:\tau$$

In [ ]:
# The complete syntax
print("The Simply Typed Lambda Calculus:")
print()
print("  TYPES:")
print("    τ ::= nat | bool | τ₁ → τ₂")
print()
print("  EXPRESSIONS:")
print("    M ::= x | λx:τ.M | M₁ M₂")
print()
print("  CONTEXTS:")
print("    Γ ::= ∅ | Γ, x:τ")

## 2. The Typing Rules

$$\frac{(x:\tau) \in \Gamma}{\Gamma \vdash x : \tau} \text{ (VAR)}$$

$$\frac{\Gamma, x:\tau_1 \vdash M : \tau_2}{\Gamma \vdash (\lambda x:\tau_1. M) : \tau_1 \to \tau_2} \text{ (ABS)}$$

$$\frac{\Gamma \vdash M : \tau_1 \to \tau_2 \quad \Gamma \vdash N : \tau_1}{\Gamma \vdash (M\ N) : \tau_2} \text{ (APP)}$$

In [ ]:
# Rule usage summary
print("The Three Typing Rules:")
print()
print("  VAR: If (x:τ) ∈ Γ, then Γ ⊢ x : τ")
print("        'Look up variables in context'")
print()
print("  ABS: If Γ,x:τ₁ ⊢ M : τ₂, then Γ ⊢ λx:τ₁.M : τ₁→τ₂")
print("        'Type lambda by extending context'")
print()
print("  APP: If Γ ⊢ M : τ₁→τ₂ and Γ ⊢ N : τ₁, then Γ ⊢ (M N) : τ₂")
print("        'Apply function to matching argument'")

# Count rule uses
rule_counts = derivations_df['rule_applied'].value_counts()
print("\n  Rule usage in our derivations:")
for rule, count in rule_counts.items():
    print(f"    {rule}: {count} uses")

## 3. The Key Theorems

| Theorem | Statement | Implication |
|---------|-----------|-------------|
| **Preservation** | $\Gamma \vdash M : \tau$ and $M \to M'$ implies $\Gamma \vdash M' : \tau$ | Types preserved during reduction |
| **Progress** | If $\emptyset \vdash M : \tau$, then $M$ is a value or can reduce | No stuck states |
| **Strong Normalization** | If $\Gamma \vdash M : \tau$, all reduction sequences from $M$ are finite | Termination guaranteed |

In [ ]:
# Verify theorems with data
print("Theorem Verification:")
print()

# All typed expressions terminate
typed = expressions_df[expressions_df['typable'] == True]
all_terminate = typed['terminates'].all()
print(f"  Strong Normalization: All {len(typed)} typed expressions terminate? {all_terminate}")

# All traces end in values
trace_endpoints = traces_df.groupby('trace_id').last()
all_values = trace_endpoints['is_value'].all()
print(f"  Progress: All {len(trace_endpoints)} traces end in values? {all_values}")

# Types preserved (implied by our data structure)
print(f"  Preservation: Types constant throughout each trace? True (by construction)")

## 4. What the Discipline Achieves

### The Omega Problem: SOLVED

Omega is untypable. Therefore, if an expression passes the typing discipline, it cannot loop forever.

### Archive Safety: GUARANTEED

The reduction engine halt of Year 800 cannot recur. All catalogued passages are verified to terminate.

In [ ]:
# The Omega problem is solved
print("The Omega Problem: SOLVED")
print()

# Find omega-related expressions
omega_related = expressions_df[
    expressions_df['notes'].str.contains('Omega|non-terminat|self-appl', case=False, na=False)
]

print("  Omega-related expressions:")
for _, row in omega_related.iterrows():
    typable = "TYPABLE" if row['typable'] else "UNTYPABLE"
    print(f"    {row['expression'][:40]}...")
    print(f"      Status: {typable}")
    print()

print("  Result: ALL non-terminating expressions are UNTYPABLE.")
print("  The typing discipline excludes them by construction.")

## 5. What the Discipline Costs

### Lost Expressiveness

- **Polymorphism**: Identity must be typed for each use
- **General recursion**: Y combinator forbidden
- **Self-reference**: No passage can take itself as argument

In [ ]:
# The costs
print("The Costs of Safety:")
print()

untypable = expressions_df[expressions_df['typable'] == False]
print(f"  Untypable expressions: {len(untypable)}")
print()

for _, row in untypable.iterrows():
    terminates = "terminates" if row['terminates'] else "non-terminating"
    print(f"  {row['expression'][:50]}...")
    print(f"    {terminates}")
    print(f"    Reason: {row['notes'][:60]}...")
    print()

## 6. The Church Encodings Survive

Kelleth Mund's Church encodings can be typed:

- **Numerals**: $(\tau \to \tau) \to \tau \to \tau$
- **Booleans**: $\tau \to \tau \to \tau$
- **Arithmetic**: SUCC, PLUS, MULT all typable

Brennis preserved his father's core work within the typed framework.

In [ ]:
# Church encodings survive
print("Kelleth's Legacy: Church Encodings")
print()

church = expressions_df[expressions_df['notes'].str.contains('Church', case=False, na=False)]

for _, row in church.head(6).iterrows():
    typable = "TYPABLE" if row['typable'] else "UNTYPABLE"
    print(f"  {row['expression'][:35]}")
    print(f"    Type: {row['type'][:40]}..." if len(str(row['type'])) > 40 else f"    Type: {row['type']}")
    print(f"    Status: {typable}")
    print()

## 7. The Complete Classification

Let's classify all expressions in our dataset.

In [ ]:
# Complete classification
print("Complete Expression Classification:")
print("=" * 50)

# Typable and terminating
safe = expressions_df[(expressions_df['typable'] == True) & (expressions_df['terminates'] == True)]
print(f"\nSAFE (typable, terminating): {len(safe)}")

# Typable but non-terminating (shouldn't exist by Strong Normalization)
impossible = expressions_df[(expressions_df['typable'] == True) & (expressions_df['terminates'] == False)]
print(f"IMPOSSIBLE (typable, non-terminating): {len(impossible)}")

# Untypable but terminating
rejected_safe = expressions_df[(expressions_df['typable'] == False) & (expressions_df['terminates'] == True)]
print(f"REJECTED BUT SAFE (untypable, terminating): {len(rejected_safe)}")

# Untypable and non-terminating
dangerous = expressions_df[(expressions_df['typable'] == False) & (expressions_df['terminates'] == False)]
print(f"DANGEROUS (untypable, non-terminating): {len(dangerous)}")

print("\nConclusion:")
print("  - ALL typable expressions terminate (Strong Normalization)")
print("  - ALL non-terminating expressions are rejected")
print("  - Some safe expressions are also rejected (conservative)")

## 8. The Mund Legacy

Two generations of Munds shaped the passage tradition:

| Scholar | Contribution | Key Insight |
|---------|--------------|-------------|
| **Kelleth Mund** (720-798) | Pure Passage Calculus | Everything reduces to abstraction and application |
| **Brennis Mund** (780-862) | Typed Passages | Boundaries prevent dissolution |

Together, they gave the Archive both the calculus and the discipline to use it safely.

In [ ]:
# The Mund Legacy
print("The Mund Legacy:")
print()
print("  KELLETH MUND (720-798)")
print("  'The Pure Passage Calculus'")
print("    - Variable binding and abstraction")
print("    - Church encodings (numerals, booleans)")
print("    - The Y combinator and fixed points")
print("    - Discovery of Omega")
print()
print("  BRENNIS MUND (780-862)")
print("  'The Typed Passage Discipline'")
print("    - Base types and arrow types")
print("    - The typing judgment Γ ⊢ M : τ")
print("    - Type safety (Preservation + Progress)")
print("    - Strong Normalization")
print()
print("  Together: A calculus AND the discipline to use it safely.")

## 9. Looking Forward

Brennis's final dedication acknowledged that simply typed lambda calculus was a beginning, not an end:

> "To those who come after — may you extend what we could only begin."

The Types & Computation tradition continues...

In [ ]:
# The path forward
print("The Types & Computation Series:")
print()
print("  Course 1: Pure Passage Calculus (Kelleth Mund)")
print("    Lambda calculus fundamentals")
print()
print("  Course 2: Combinatorial Reduction (Jorell Vance)")
print("    Variable-free computation with S, K, I")
print()
print("  Course 3: Typed Passages (Brennis Mund) [THIS COURSE]")
print("    Simply typed lambda calculus")
print()
print("  Course 4: Continuous Domains (Arren Mott)")
print("    Domain theory and fixed-point semantics")
print()
print("  Course 5: Dependent Classifications (Sereth Linn)")
print("    Dependent types — types that depend on values")
print()
print("  Course 6: Resource Typing (Quellen Vast)")
print("    Linear types — track resource usage")
print()
print("  Course 7: Equivalence via Passage (Dora Keth)")
print("    Homotopy type theory — univalence")

## Summary

In this course, we learned Brennis Mund's **Typed Passage Discipline**:

### The Problem
- Omega and other non-terminating expressions threaten Archive stability
- Kelleth Mund discovered the problem but couldn't solve it

### The Solution
- **Types as boundaries**: Passages declare what they accept
- **Base types** (nat, bool) and **arrow types** (τ₁ → τ₂)
- **Typing judgment** Γ ⊢ M : τ with three rules (VAR, ABS, APP)

### The Guarantees
- **Preservation**: Types are preserved during reduction
- **Progress**: Well-typed terms don't get stuck
- **Strong Normalization**: Every well-typed term terminates

### The Tradeoffs
- Lost polymorphism (identity must be specialized)
- Lost general recursion (Y combinator forbidden)
- Recovered via primitive recursion

### The Legacy
Brennis gave passages their boundaries. Within those boundaries, the Archive is safe.

---

## Final Exercises

1. Write out the complete typing derivation for composition: $\lambda f.\lambda g.\lambda x.(f\ (g\ x))$

2. Classify these expressions as typable/untypable, terminating/non-terminating:
   - $(\lambda x.x)(\lambda y.y)$
   - $(\lambda x.x\ x)(\lambda y.y\ y)$
   - $\lambda f.(f\ (f\ (f\ x)))$

3. Explain in your own words why types prevent Omega.

4. If you could add ONE feature to simply typed lambda calculus, what would it be and why?

---

*End of Course 3: Typed Passages*

*Next in the series: Course 4 - Continuous Domains (Arren Mott)*